In [11]:
import duckdb
from kingsleague import *

In [12]:
import duckdb

# Nom du fichier de la base de données
db_file = 'KINGSLEAGUEfrance.db'

# Nom du fichier SQL contenant la création des tables
sql_file = 'test.sql'

# Se connecter à la base de données (crée le fichier si inexistant)
conn = duckdb.connect(db_file)

try:
    with open(sql_file, 'r') as f:
        sql_script = f.read()

    # Exécuter le script SQL
    conn.execute(sql_script)
    conn.commit()
    print(f"Le script SQL '{sql_file}' a été exécuté avec succès.")

except FileNotFoundError:
    print(f"Erreur : Le fichier SQL '{sql_file}' n'a pas été trouvé.")
except duckdb.Error as e:
    print(f"Une erreur est survenue lors de l'exécution du script SQL : {e}")

finally:
    # Fermer la connexion
    if conn:
        conn.close()

Le script SQL 'test.sql' a été exécuté avec succès.


In [13]:
equipes_et_presidents = {
    "360 Nation": ["Tchouameni", "Koundé", "Maignan", "Kone", "Mbeumo", "Mayulu"],
    "F2R": ["Aminematue", "Samir Nasri", "Jérémy Ménez"],
    "FC SILMI": ["Domingo"],
    "Generation Seven": ["Michou"],
    "KARASU": ["Kameto"],
    "PANAM ALL STARZ": ["Pfut"],
    "Unit3d": ["Squeezie", "Djilsi", "Maxime Biaggi"],
    "Wolf Pack" : ["Adil Rami"]
}
# Se connecter à la base de données
conn = duckdb.connect(db_file)

try:
    for nom_equipe in equipes_et_presidents:
        conn.execute("INSERT INTO equipe VALUES (nextval('seq_equipe_id'), ?)", (nom_equipe,))
        print(f"L'équipe '{nom_equipe}' a été ajoutée avec succès.")

    conn.commit()

except duckdb.Error as e:
    print(f"Une erreur est survenue lors de l'insertion : {e}")

finally:
    if conn:
        conn.close()

L'équipe '360 Nation' a été ajoutée avec succès.
L'équipe 'F2R' a été ajoutée avec succès.
L'équipe 'FC SILMI' a été ajoutée avec succès.
L'équipe 'Generation Seven' a été ajoutée avec succès.
L'équipe 'KARASU' a été ajoutée avec succès.
L'équipe 'PANAM ALL STARZ' a été ajoutée avec succès.
L'équipe 'Unit3d' a été ajoutée avec succès.
L'équipe 'Wolf Pack' a été ajoutée avec succès.


In [14]:
conn = duckdb.connect(db_file)

try:
    for equipe, presidents in equipes_et_presidents.items():
        # Récupérer l'equipe_id à partir du nom de l'équipe
        result = conn.execute("SELECT equipe_id FROM equipe WHERE nom = ?", (equipe,)).fetchone()
        if result is None:
            print(f"⚠️ Équipe '{equipe}' introuvable dans la base.")
            continue

        equipe_id = result[0]

        # Insérer les présidents associés à cette équipe
        for president in presidents:
            conn.execute(
                "INSERT INTO president VALUES (nextval('seq_president_id'), ?, ?)",
                (president, equipe_id)
            )
            print(f"✅ Président '{president}' ajouté à l'équipe '{equipe}' (ID {equipe_id}).")

    conn.commit()

except duckdb.Error as e:
    print(f"❌ Une erreur est survenue : {e}")

finally:
    if conn:
        conn.close()

✅ Président 'Tchouameni' ajouté à l'équipe '360 Nation' (ID 1).
✅ Président 'Koundé' ajouté à l'équipe '360 Nation' (ID 1).
✅ Président 'Maignan' ajouté à l'équipe '360 Nation' (ID 1).
✅ Président 'Kone' ajouté à l'équipe '360 Nation' (ID 1).
✅ Président 'Mbeumo' ajouté à l'équipe '360 Nation' (ID 1).
✅ Président 'Mayulu' ajouté à l'équipe '360 Nation' (ID 1).
✅ Président 'Aminematue' ajouté à l'équipe 'F2R' (ID 2).
✅ Président 'Samir Nasri' ajouté à l'équipe 'F2R' (ID 2).
✅ Président 'Jérémy Ménez' ajouté à l'équipe 'F2R' (ID 2).
✅ Président 'Domingo' ajouté à l'équipe 'FC SILMI' (ID 3).
✅ Président 'Michou' ajouté à l'équipe 'Generation Seven' (ID 4).
✅ Président 'Kameto' ajouté à l'équipe 'KARASU' (ID 5).
✅ Président 'Pfut' ajouté à l'équipe 'PANAM ALL STARZ' (ID 6).
✅ Président 'Squeezie' ajouté à l'équipe 'Unit3d' (ID 7).
✅ Président 'Djilsi' ajouté à l'équipe 'Unit3d' (ID 7).
✅ Président 'Maxime Biaggi' ajouté à l'équipe 'Unit3d' (ID 7).
✅ Président 'Adil Rami' ajouté à l'équipe 

In [35]:
def inserer_joueurs(nom_equipe, joueurs, db_file='kingsleaguefrance.db'):
    """
    Insère une liste de joueurs dans la base de données DuckDB.

    :param nom_equipe: Nom de l'équipe associée aux joueurs
    :param joueurs: Liste de tuples (nom, poste, note, wildcard)
    :param db_file: Fichier de base de données DuckDB
    """
    conn = duckdb.connect(db_file)

    try:
        # Récupérer l'equipe_id à partir du nom de l'équipe
        result = conn.execute("SELECT equipe_id FROM equipe WHERE nom = ?", (nom_equipe,)).fetchone()

        
        if result is None:
            print(f"⚠️ Équipe '{nom_equipe}' introuvable.")
            return
        
        equipe_id = result[0]

        # Insérer les joueurs un par un
        for nom, poste, note, wildcard in joueurs:
            conn.execute(
                """
                INSERT INTO joueur (joueur_id, nom, equipe_id, poste, note, wildcard)
                VALUES (nextval('seq_joueur_id'), ?, ?, ?, ?, ?)
                """,
                (nom, equipe_id, poste, note, wildcard)
            )
            print(f"✅ Joueur '{nom}' inséré (poste={poste}, note={note}, wildcard={wildcard})")

        conn.commit()

    except duckdb.Error as e:
        print(f"❌ Erreur lors de l'insertion : {e}")

    finally:
        conn.close()


In [38]:
joueurs_360Nation = [
    ("Felice Maestro", "Entraîneur", None, False),
    ("Yann Mvita", "Défenseur", 78, False),
    ("Christophe Rodrigues Silva", "Attaquant", 75, False),
    ("Marco Alarcon", "Défenseur", 79, False),
    ("Ilian Hamoum", "Milieu de terrain", 71, False),
    ("Mohamed Tbahriti", "Attaquant", 77, False),
    ("JMK", "Gardien de but", 84, False),
    ("Assilenou Toviave", "Défenseur", 75, False),
    ("Bilel Boucheker", "Attaquant", 77, False),
    ("Leny Gomez", "Attaquant", 82, False),
    ("Hamed Karamoko", "Milieu de terrain", 81, False),
    ("Landry N'Gala", "Attaquant", None, True),
    ("Daniel Mendy", "Milieu de terrain", None, True),
    ("Skander Si Chaib", "Milieu de terrain", None, True)
]

inserer_joueurs("360 Nation", joueurs_360Nation)

✅ Joueur 'Felice Maestro' inséré (poste=Entraîneur, note=None, wildcard=False)
✅ Joueur 'Yann Mvita' inséré (poste=Défenseur, note=78, wildcard=False)
✅ Joueur 'Christophe Rodrigues Silva' inséré (poste=Attaquant, note=75, wildcard=False)
✅ Joueur 'Marco Alarcon' inséré (poste=Défenseur, note=79, wildcard=False)
✅ Joueur 'Ilian Hamoum' inséré (poste=Milieu de terrain, note=71, wildcard=False)
✅ Joueur 'Mohamed Tbahriti' inséré (poste=Attaquant, note=77, wildcard=False)
✅ Joueur 'JMK' inséré (poste=Gardien de but, note=84, wildcard=False)
✅ Joueur 'Assilenou Toviave' inséré (poste=Défenseur, note=75, wildcard=False)
✅ Joueur 'Bilel Boucheker' inséré (poste=Attaquant, note=77, wildcard=False)
✅ Joueur 'Leny Gomez' inséré (poste=Attaquant, note=82, wildcard=False)
✅ Joueur 'Hamed Karamoko' inséré (poste=Milieu de terrain, note=81, wildcard=False)
✅ Joueur 'Landry N'Gala' inséré (poste=Attaquant, note=None, wildcard=True)
✅ Joueur 'Daniel Mendy' inséré (poste=Milieu de terrain, note=None, 

In [39]:
joueurs_F2R = [
    ("Samir Nasri", "Entraîneur", None, False),
    ("Amara Fofana", "Attaquant", 76, False),
    ("Sahir Boumhand", "Défenseur", 83, False),
    ("Louka Masset", "Défenseur", 80, False),
    ("Allan Rakotovazaha", "Gardien de but", 83, False),
    ("Safwane Ben Hariz", "Défenseur", 83, False),
    ("Andy Wothor", "Attaquant", 83, False),
    ("Haris El Mouttaqi", "Attaquant", 81, False),
    ("Sofiane Chenaa", "Milieu de terrain", 78, False),
    ("Rayan Tahar", "Défenseur", 78, False),
    ("William Harhouz", "Attaquant", 78, False),
    ("Kevin Bru", "Milieu de terrain", 83, False)
]
inserer_joueurs("F2R", joueurs_F2R)

✅ Joueur 'Samir Nasri' inséré (poste=Entraîneur, note=None, wildcard=False)
✅ Joueur 'Amara Fofana' inséré (poste=Attaquant, note=76, wildcard=False)
✅ Joueur 'Sahir Boumhand' inséré (poste=Défenseur, note=83, wildcard=False)
✅ Joueur 'Louka Masset' inséré (poste=Défenseur, note=80, wildcard=False)
✅ Joueur 'Allan Rakotovazaha' inséré (poste=Gardien de but, note=83, wildcard=False)
✅ Joueur 'Safwane Ben Hariz' inséré (poste=Défenseur, note=83, wildcard=False)
✅ Joueur 'Andy Wothor' inséré (poste=Attaquant, note=83, wildcard=False)
✅ Joueur 'Haris El Mouttaqi' inséré (poste=Attaquant, note=81, wildcard=False)
✅ Joueur 'Sofiane Chenaa' inséré (poste=Milieu de terrain, note=78, wildcard=False)
✅ Joueur 'Rayan Tahar' inséré (poste=Défenseur, note=78, wildcard=False)
✅ Joueur 'William Harhouz' inséré (poste=Attaquant, note=78, wildcard=False)
✅ Joueur 'Kevin Bru' inséré (poste=Milieu de terrain, note=83, wildcard=False)


In [37]:
import duckdb

# 'wildcard' est défini sur True ou False selon le cas
joueurs_FCSILMI = [
    ("Maktoum Abdelhack", "Entraîneur", None, False),
    ("Leandro Fernandes", "Gardien de but", 74, False),
    ("Allan Eloundou Mani", "Milieu de terrain", 70, False),
    ("Clément Michelet", "Milieu de terrain", 82, False),
    ("Jhon Suarez", "Milieu de terrain", 78, False),
    ("Lukas Jeremie", "Défenseur", 70, False),
    ("Nicolas Le Metayer", "Défenseur", 78, False),
    ("Maxime Dague", "Gardien de but", 83, False),
    ("Bilal Hadraoui", "Défenseur", 71, False),
    ("Abdel-Amine Tran", "Milieu de terrain", 76, False),
    ("Liam Mirandon", "Attaquant", 83, False),
    # Joueurs wildcard
    ("Guilherme Carvalho Feitosa", "Milieu de terrain", None, True),
    ("André Nascimento de Oliveira", "Attaquant", None, True),
    ("Adama Dhee", "Attaquant", None, True)
]

inserer_joueurs("FC SILMI", joueurs_FCSILMI)

✅ Joueur 'Maktoum Abdelhack' inséré (poste=Entraîneur, note=None, wildcard=False)
✅ Joueur 'Leandro Fernandes' inséré (poste=Gardien de but, note=74, wildcard=False)
✅ Joueur 'Allan Eloundou Mani' inséré (poste=Milieu de terrain, note=70, wildcard=False)
✅ Joueur 'Clément Michelet' inséré (poste=Milieu de terrain, note=82, wildcard=False)
✅ Joueur 'Jhon Suarez' inséré (poste=Milieu de terrain, note=78, wildcard=False)
✅ Joueur 'Lukas Jeremie' inséré (poste=Défenseur, note=70, wildcard=False)
✅ Joueur 'Nicolas Le Metayer' inséré (poste=Défenseur, note=78, wildcard=False)
✅ Joueur 'Maxime Dague' inséré (poste=Gardien de but, note=83, wildcard=False)
✅ Joueur 'Bilal Hadraoui' inséré (poste=Défenseur, note=71, wildcard=False)
✅ Joueur 'Abdel-Amine Tran' inséré (poste=Milieu de terrain, note=76, wildcard=False)
✅ Joueur 'Liam Mirandon' inséré (poste=Attaquant, note=83, wildcard=False)
✅ Joueur 'Guilherme Carvalho Feitosa' inséré (poste=Milieu de terrain, note=None, wildcard=True)
✅ Joueur '

In [41]:
joueurs_G7 = [
    ("Djibril Cissé", "Entraîneur", None, False),
    ("Yllan Ait-Ouarab", "Milieu de terrain", 75, False),
    ("Mehdi Tenniche", "Défenseur", 73, False),
    ("Brunnel Tutuana Theun Tatuom", "Défenseur", 78, False),
    ("Mamoudou Cissokho", "Attaquant", 77, False),
    ("Clement Goguey", "Milieu de terrain", 82, False),
    ("Solal Pelmard-Bassi", "Gardien de but", 76, False),
    ("Enzo Dron", "Milieu de terrain", 82, False),
    ("Nyls Gomez", "Milieu de terrain", 71, False),
    ("Yonni Bouguerra", "Gardien de but", 80, False),
    ("Driss Zidane", "Milieu de terrain", 76, False),
    ("Hugo Rubio", None, None, True),
    ("Paul-Georges Ntep", None, None, True),
    ("Yacouba Sylla", None, None, True)
]
inserer_joueurs("Generation Seven", joueurs_G7)

✅ Joueur 'Djibril Cissé' inséré (poste=Entraîneur, note=None, wildcard=False)
✅ Joueur 'Yllan Ait-Ouarab' inséré (poste=Milieu de terrain, note=75, wildcard=False)
✅ Joueur 'Mehdi Tenniche' inséré (poste=Défenseur, note=73, wildcard=False)
✅ Joueur 'Brunnel Tutuana Theun Tatuom' inséré (poste=Défenseur, note=78, wildcard=False)
✅ Joueur 'Mamoudou Cissokho' inséré (poste=Attaquant, note=77, wildcard=False)
✅ Joueur 'Clement Goguey' inséré (poste=Milieu de terrain, note=82, wildcard=False)
✅ Joueur 'Solal Pelmard-Bassi' inséré (poste=Gardien de but, note=76, wildcard=False)
✅ Joueur 'Enzo Dron' inséré (poste=Milieu de terrain, note=82, wildcard=False)
✅ Joueur 'Nyls Gomez' inséré (poste=Milieu de terrain, note=71, wildcard=False)
✅ Joueur 'Yonni Bouguerra' inséré (poste=Gardien de but, note=80, wildcard=False)
✅ Joueur 'Driss Zidane' inséré (poste=Milieu de terrain, note=76, wildcard=False)
✅ Joueur 'Hugo Rubio' inséré (poste=None, note=None, wildcard=True)
✅ Joueur 'Paul-Georges Ntep' i

In [42]:
joueurs_KARASU= [
    ("Sabry Bezahaf", "Entraîneur", None, False),
    ("Yacine Ben Cherki", "Gardien de but", 75, False),
    ("Salim Abdou Mohamed", "Attaquant", 76, False),
    ("Alexandre Fakir", "Milieu de terrain", 77, False),
    ("Zinedine Ayadi", "Milieu de terrain", 77, False),
    ("Abdelhamid Touati", "Défenseur", 78, False),
    ("Ahmed Ouachani", "Milieu de terrain", 77, False),
    ("Mehdi Gacem", "Défenseur", 76, False),
    ("Mohammed Berrabah", "Défenseur", 84, False),
    ("Kevin Kuya", "Milieu de terrain", 75, False),
    ("Bilal Bouchelouche", "Gardien de but", 79, False),
    ("Mehdi Marin", "Attaquant", None, True),
    ("Hicham Rougab", "Attaquant", None, True),
    ("Mounir Gagui", "Milieu de terrain", None, True)
]

inserer_joueurs("KARASU", joueurs_KARASU)

✅ Joueur 'Sabry Bezahaf' inséré (poste=Entraîneur, note=None, wildcard=False)
✅ Joueur 'Yacine Ben Cherki' inséré (poste=Gardien de but, note=75, wildcard=False)
✅ Joueur 'Salim Abdou Mohamed' inséré (poste=Attaquant, note=76, wildcard=False)
✅ Joueur 'Alexandre Fakir' inséré (poste=Milieu de terrain, note=77, wildcard=False)
✅ Joueur 'Zinedine Ayadi' inséré (poste=Milieu de terrain, note=77, wildcard=False)
✅ Joueur 'Abdelhamid Touati' inséré (poste=Défenseur, note=78, wildcard=False)
✅ Joueur 'Ahmed Ouachani' inséré (poste=Milieu de terrain, note=77, wildcard=False)
✅ Joueur 'Mehdi Gacem' inséré (poste=Défenseur, note=76, wildcard=False)
✅ Joueur 'Mohammed Berrabah' inséré (poste=Défenseur, note=84, wildcard=False)
✅ Joueur 'Kevin Kuya' inséré (poste=Milieu de terrain, note=75, wildcard=False)
✅ Joueur 'Bilal Bouchelouche' inséré (poste=Gardien de but, note=79, wildcard=False)
✅ Joueur 'Mehdi Marin' inséré (poste=Attaquant, note=None, wildcard=True)
✅ Joueur 'Hicham Rougab' inséré (p

In [43]:
joueurs_PANAME_ALLSTAR = [
    ("Saïd Dorbani", "Entraîneur", None, False),
    ("Phi-Long Tran", "Attaquant", 83, False),
    ("Nzaba Lungituka", "Défenseur", 78, False),
    ("Adama Wagui", "Gardien de but", 73, False),
    ("Daniel Mboudou", "Attaquant", 83, False),
    ("Idir Ahmin", "Milieu de terrain", 77, False),
    ("Louis Coppin", "Attaquant", 78, False),
    ("Guillaume Lesec", "Gardien de but", 79, False),
    ("Alric Francisco", "Défenseur", 76, False),
    ("Nicolas Martins", "Milieu de terrain", 83, False),
    ("José Pereira Do Amaral", "Défenseur", 77, False),
    ("Harrison Manzala", "Milieu de terrain", None, True),
    ("Nassim L'Ghoul", None, None, True),
    ("Sofiane Bendaoud", None, None, True)
]
inserer_joueurs("PANAM ALL STARZ", joueurs_PANAME_ALLSTAR)


✅ Joueur 'Saïd Dorbani' inséré (poste=Entraîneur, note=None, wildcard=False)
✅ Joueur 'Phi-Long Tran' inséré (poste=Attaquant, note=83, wildcard=False)
✅ Joueur 'Nzaba Lungituka' inséré (poste=Défenseur, note=78, wildcard=False)
✅ Joueur 'Adama Wagui' inséré (poste=Gardien de but, note=73, wildcard=False)
✅ Joueur 'Daniel Mboudou' inséré (poste=Attaquant, note=83, wildcard=False)
✅ Joueur 'Idir Ahmin' inséré (poste=Milieu de terrain, note=77, wildcard=False)
✅ Joueur 'Louis Coppin' inséré (poste=Attaquant, note=78, wildcard=False)
✅ Joueur 'Guillaume Lesec' inséré (poste=Gardien de but, note=79, wildcard=False)
✅ Joueur 'Alric Francisco' inséré (poste=Défenseur, note=76, wildcard=False)
✅ Joueur 'Nicolas Martins' inséré (poste=Milieu de terrain, note=83, wildcard=False)
✅ Joueur 'José Pereira Do Amaral' inséré (poste=Défenseur, note=77, wildcard=False)
✅ Joueur 'Harrison Manzala' inséré (poste=Milieu de terrain, note=None, wildcard=True)
✅ Joueur 'Nassim L'Ghoul' inséré (poste=None, no

In [44]:
joueurs_united = [
    ("Campi Grégory", "Entraîneur", None, False),
    ("Marvin Emmanuel", "Attaquant", 82, False),
    ("Alseny Sano", "Défenseur", 75, False),
    ("Abdraman Toure", "Attaquant", 82, False),
    ("Van Landry Nguekam", "Défenseur", 71, False),
    ("Louis Lapouge", "Attaquant", 72, False),
    ("Yassine Skira", "Milieu de terrain", 72, False),
    ("Florian Forestier", "Gardien de but", 83, False),
    ("Théo Chendri", "Milieu de terrain", 83, False),
    ("Alex Diliberto", "Milieu de terrain", 71, False),
    ("Youssef Khatiri", "Défenseur", 72, False),
    ("Baptiste Macon", "Milieu de terrain", None, True),
    ("Anthony SCARAMOZZINO", "Défenseur", None, True),
    ("Emmanuel Mifsud", "Gardien de but", None, True)
]

inserer_joueurs("Unit3d", joueurs_united)

✅ Joueur 'Campi Grégory' inséré (poste=Entraîneur, note=None, wildcard=False)
✅ Joueur 'Marvin Emmanuel' inséré (poste=Attaquant, note=82, wildcard=False)
✅ Joueur 'Alseny Sano' inséré (poste=Défenseur, note=75, wildcard=False)
✅ Joueur 'Abdraman Toure' inséré (poste=Attaquant, note=82, wildcard=False)
✅ Joueur 'Van Landry Nguekam' inséré (poste=Défenseur, note=71, wildcard=False)
✅ Joueur 'Louis Lapouge' inséré (poste=Attaquant, note=72, wildcard=False)
✅ Joueur 'Yassine Skira' inséré (poste=Milieu de terrain, note=72, wildcard=False)
✅ Joueur 'Florian Forestier' inséré (poste=Gardien de but, note=83, wildcard=False)
✅ Joueur 'Théo Chendri' inséré (poste=Milieu de terrain, note=83, wildcard=False)
✅ Joueur 'Alex Diliberto' inséré (poste=Milieu de terrain, note=71, wildcard=False)
✅ Joueur 'Youssef Khatiri' inséré (poste=Défenseur, note=72, wildcard=False)
✅ Joueur 'Baptiste Macon' inséré (poste=Milieu de terrain, note=None, wildcard=True)
✅ Joueur 'Anthony SCARAMOZZINO' inséré (poste=

In [45]:
joueurs_wolfpack = [
    ("Yohan Di Tommaso", "Entraîneur", None, False),
    ("Jordan Boli", "Défenseur", 83, False),
    ("Quentin Bakouzou", "Attaquant", 77, False),
    ("Mathias Faillenot", "Défenseur", 77, False),
    ("Mourad Khalfa", "Milieu de terrain", 78, False),
    ("Dylan Djedor", "Milieu de terrain", 72, False),
    ("Oscar Le Guillou", "Attaquant", 75, False),
    ("Lotfi Djellaili", "Milieu de terrain", 71, False),
    ("Alexandre Delarboulas", "Défenseur", 85, False),
    ("Thibaut Pereira", "Gardien de but", 77, False),
    ("Beendo Z", "Attaquant", 78, False),
    ("Leo Colin", "Gardien de but", None, True),
    ("Adrien BARD", "Milieu de terrain", None, True),
    ("Jill Berardo", "Attaquant", None, True)
]

inserer_joueurs("Wolf Pack", joueurs_wolfpack)


✅ Joueur 'Yohan Di Tommaso' inséré (poste=Entraîneur, note=None, wildcard=False)
✅ Joueur 'Jordan Boli' inséré (poste=Défenseur, note=83, wildcard=False)
✅ Joueur 'Quentin Bakouzou' inséré (poste=Attaquant, note=77, wildcard=False)
✅ Joueur 'Mathias Faillenot' inséré (poste=Défenseur, note=77, wildcard=False)
✅ Joueur 'Mourad Khalfa' inséré (poste=Milieu de terrain, note=78, wildcard=False)
✅ Joueur 'Dylan Djedor' inséré (poste=Milieu de terrain, note=72, wildcard=False)
✅ Joueur 'Oscar Le Guillou' inséré (poste=Attaquant, note=75, wildcard=False)
✅ Joueur 'Lotfi Djellaili' inséré (poste=Milieu de terrain, note=71, wildcard=False)
✅ Joueur 'Alexandre Delarboulas' inséré (poste=Défenseur, note=85, wildcard=False)
✅ Joueur 'Thibaut Pereira' inséré (poste=Gardien de but, note=77, wildcard=False)
✅ Joueur 'Beendo Z' inséré (poste=Attaquant, note=78, wildcard=False)
✅ Joueur 'Leo Colin' inséré (poste=Gardien de but, note=None, wildcard=True)
✅ Joueur 'Adrien BARD' inséré (poste=Milieu de te